'''
 * Copyright (c) 2008 Radhamadhab Dalai
 *
 * Permission is hereby granted, free of charge, to any person obtaining a copy
 * of this software and associated documentation files (the "Software"), to deal
 * in the Software without restriction, including without limitation the rights
 * to use, copy, modify, merge, publish, distribute, sublicense, and/or sell
 * copies of the Software, and to permit persons to whom the Software is
 * furnished to do so, subject to the following conditions:
 *
 * The above copyright notice and this permission notice shall be included in
 * all copies or substantial portions of the Software.
 *
 * THE SOFTWARE IS PROVIDED "AS IS", WITHOUT WARRANTY OF ANY KIND, EXPRESS OR
 * IMPLIED, INCLUDING BUT NOT LIMITED TO THE WARRANTIES OF MERCHANTABILITY,
 * FITNESS FOR A PARTICULAR PURPOSE AND NONINFRINGEMENT. IN NO EVENT SHALL THE
 * AUTHORS OR COPYRIGHT HOLDERS BE LIABLE FOR ANY CLAIM, DAMAGES OR OTHER
 * LIABILITY, WHETHER IN AN ACTION OF CONTRACT, TORT OR OTHERWISE, ARISING FROM,
 * OUT OF OR IN CONNECTION WITH THE SOFTWARE OR THE USE OR OTHER DEALINGS IN
 * THE SOFTWARE.
'''

## Monte Carlo EM Algorithm (MCEM)

## Introduction

A common difficulty with the implementation of the EM algorithm is that each "E-step" requires the computation of the expected log-likelihood, $ Q(\theta | \theta^{(t)}, x) $. Wei and Tanner (1990a, b) propose the Monte Carlo EM (MCEM) approach to address this challenge. This method involves simulating $ Z_1, \dots, Z_m $ from the conditional distribution $ p(z | x, \theta) $ and then maximizing the approximate complete-data log-likelihood:

$$
Q_m(\theta | x) = \frac{1}{m} \sum_{i=1}^m \log L_c(\theta | x, z_i),
\tag{5.20}
$$

where $ L_c(\theta | x, z) $ represents the complete-data likelihood.

As $ m \to \infty $, $ Q_m(\theta | x) $ converges to $ Q(\theta | \theta^{(t)}, x) $, and the MCEM algorithm approaches the regular EM algorithm. The authors suggest increasing $ m $ along with the iterations. While maximizing a sum like $ Q_m(\theta | x) $ can be complex, closed-form solutions are often available in exponential family settings.

---

## Example 5.20: MCEM for Censored Data

The EM solution for censored data (Example 5.17) can easily transition into an MCEM solution. For the EM sequence:

$$
\theta^{(t+1)} = \frac{n_m}{T_m} + \frac{1}{n_m} \mathbb{E}_{\theta^{(t)}}[Z_i | x],
$$

the MCEM solution replaces \( \mathbb{E}_{\theta^{(t)}}[Z_i | x] \) with:

$$
\frac{1}{M} \sum_{i=1}^M Z_i, \quad Z_i \sim p(z | x, \theta^{(t)}).
$$

The MCEM sequence is shown in the right panel of Figure 5.6. While the convergence of MCEM is slower than EM, the variability is controlled by the choice of $ M $. Increasing $ M $ brings the sequences closer together.

---

## Example 21: Genetic Linkage

A classic example of the EM algorithm is the genetics problem (see Rao 1973, Dempster et al. 1977, Tanner 1996), where observations $ (n_1, n_2, n_3, n_4) $ are drawn from the multinomial distribution:

$$
\text{Multinomial}\left(n; \frac{\theta}{2}, \frac{\theta}{2}, \frac{1-\theta}{2}, \frac{1-\theta}{2}\right).
$$

Estimation becomes easier if the first cell is split into two cells, creating the augmented model:

$$
(n_{11}, n_{12}, n_2, n_3, n_4) \sim \text{Multinomial}\left(n; \frac{\theta}{4}, \frac{\theta}{4}, \frac{\theta}{2}, \frac{1-\theta}{2}, \frac{1-\theta}{2}\right).
$$

This augmentation allows for simpler computation of the expected log-likelihood and facilitates the iterative application of the EM algorithm.
# Monte Carlo EM Algorithm (MCEM)

## Genetic Linkage (Continued)

The complete-data likelihood function is:

$$
L_c(\theta | x, z) = \theta^{z_1 + z_2} (1 - \theta)^{z_3 + z_4},
$$

as opposed to the observed-data likelihood function:

$$
L(\theta | x) = (\theta / 2)^{z_1 + z_2} ((1 - \theta) / 2)^{z_3 + z_4}.
$$

The expected complete log-likelihood function is:

$$
\mathbb{E}_{\theta^{(t)}} \left[ (Z_1 + Z_2) \log \theta + (Z_3 + Z_4) \log (1 - \theta) \right].
$$

Simplifying:

$$
\mathbb{E}_{\theta^{(t)}} \left[ (Z_1 + Z_2) \right] = \frac{z_1 \theta}{2 + \theta}, \quad 
\mathbb{E}_{\theta^{(t)}} \left[ (Z_3 + Z_4) \right] = z_3 + z_4,
$$

we have:

$$
Q(\theta | \theta^{(t)}) = \frac{z_1 \theta}{2 + \theta} \log \theta + (z_3 + z_4) \log (1 - \theta).
$$

Maximizing this leads to:

$$
\theta^{(t+1)} = \frac{\mathbb{E}_{\theta^{(t)}}[Z_1 + Z_2] + z_3 + z_4}{z_1 + z_2 + z_3 + z_4}.
$$

If we instead use the Monte Carlo EM algorithm, $ \mathbb{E}_{\theta^{(t)}}[Z_1 + Z_2] $ is replaced with the average:

$$
\bar{Z}_m = \frac{1}{m} \sum_{i=1}^m Z_i,
$$

where the $ Z_i $ are simulated from the binomial distribution $ B(z_1, \theta^{(t)}/(2 + \theta^{(t)})) $. The maximum becomes:

$$
\theta^{(t+1)} = \frac{\bar{Z}_m + z_3 + z_4}{\bar{Z}_m + z_1 + z_2 + z_3 + z_4}.
$$

This example illustrates the Monte Carlo EM algorithm, though the regular EM algorithm also applies.

---

## Example 22: Capture-Recapture Models Revisited

A generalization of a capture-recapture model assumes that an animal $ i $, $ i = 1, 2, \ldots, n $, can be captured at time $ j $, $ j = 1, 2, \ldots, t $, in one of $ m $ locations. The location is a multinomial random variable:

$$
H \sim \text{Multinomial}(\theta_1, \ldots, \theta_m).
$$

If an animal is at location $ k $, the random variable $ X \sim B(p_k) $, where $ p_k $ is the probability of capturing the animal in location $ k $.

### Example Realization

For $ t = 6 $, a typical realization for an animal might be:

$$
h = (4, 1, -, 3, 3, -), \quad x = (1, 1, 0, 1, 1, 0),
$$

where $ h $ represents the location (or lack thereof) and $ x $ represents whether the animal was captured at each time point.

### Monte Carlo EM Application

The MCEM algorithm is particularly useful here, as the expectation in the E-step is computationally intensive due to the latent variables $ h $. The MCEM replaces the expectation with simulated averages, enabling iterative updates for the parameters $ \theta_k $ and $ p_k $. See Dupuis (1995) for details on the implementation.

## Capture-Recapture Models and MCEM Algorithm

## Problem Description

In a capture-recapture model, $ h $ denotes the sequence of observed locations and non-captures for animals. This leads to a missing data problem. If all of $ h $ were observed, the maximum likelihood estimation (MLE) would be trivial, with the MLEs being the cell means.

### Random Variables

- $ X_{ijk} = 1 $ if animal $ i $ is captured at time $ j $ in location $ k $, and $ 0 $ otherwise.
- $ Y_{ijk} = I(H_{ijk} = k)I(X_{ijk} = 1) $ (the observed data).
- $ Z_{ijk} = I(H_{ijk} = k)I(X_{ijk} = 0) $ (the missing data).

### Likelihood Function

The likelihood function is given by:

$$
L(\theta_1, \dots, \theta_m, p_1, \dots, p_m | x) = \sum_{z} L(\theta_1, \dots, \theta_m, p_1, \dots, p_m | Y, X, Z),
$$

where the complete-data likelihood is:

$$
L(\theta_1, \dots, \theta_m, p_1, \dots, p_m | Y, X, Z) = \prod_{i=1}^n \prod_{j=1}^t \prod_{k=1}^m \theta_k^{Y_{ijk} + Z_{ijk}} p_k^{Y_{ijk}} (1 - p_k)^{Z_{ijk}}.
$$

The sum over $ z $ represents the expectation over all the states \( h \) that could have been visited.

---

## MCEM Algorithm for Capture-Recapture

Since directly computing the expectation in the E-step is complicated, we use an EM strategy combined with MCEM for the expectation calculation.

### Algorithm A.21: Capture-Recapture MCEM Algorithm

1. **M-step**  
   Update the probabilities $ \theta_k $ as:
   $$
   \theta_k = \frac{\sum_{i=1}^n \sum_{j=1}^t (Y_{ijk} + Z_{ijk})}{\sum_{i=1}^n \sum_{j=1}^t \sum_{k=1}^m (Y_{ijk} + Z_{ijk})}.
   $$

2. **Monte Carlo E-step**  
   If $ Z_{ijk} = 0 $, for $ i = 1, \dots, n $, generate:
   $$
   Z_{ijk} \sim \text{Multinomial}(\theta_k).
   $$
   Then calculate:
   $$
   \theta_k = \frac{\sum_{i=1}^n \sum_{j=1}^t Y_{ijk}}{n}.
   $$

---

## Notes and Insights

- Scherrer (1997) examines the performance of generalized versions of this algorithm and demonstrates that they outperform the conditional likelihood approach of Brownie et al. (1993).
- While the MCEM algorithm offers flexibility, it does not preserve the EM monotonicity property and may encounter smoothness issues when the sample size used for Monte Carlo approximations is small.


In [1]:
import numpy as np
from scipy.stats import multinomial

# Simulated data: observed captures (Y) and initial parameters
n = 10  # Number of animals
t = 6   # Number of time points
m = 3   # Number of locations

# Example observed data: random initialization for demonstration
np.random.seed(42)
Y = np.random.randint(0, 2, (n, t, m))  # Observed data matrix (n x t x m)

# Initial parameter guesses
theta = np.random.dirichlet(np.ones(m))  # Initial probabilities for locations
p = np.random.uniform(0.5, 0.9, m)       # Initial capture probabilities

# MCEM algorithm parameters
iterations = 50
sample_size = 1000

def capture_recapture_mcem(Y, theta, p, iterations, sample_size):
    n, t, m = Y.shape
    
    for iter in range(iterations):
        # Step 1: Monte Carlo E-step
        # Simulate missing data Z using multinomial distributions
        Z = np.zeros_like(Y, dtype=float)
        for i in range(n):
            for j in range(t):
                observed = Y[i, j, :]  # Observed captures
                remaining_prob = theta * (1 - p)  # Probability of non-capture
                norm_factor = np.sum(remaining_prob)
                if norm_factor > 0:
                    Z[i, j, :] = multinomial.rvs(sample_size, remaining_prob / norm_factor)
        
        # Step 2: M-step
        # Update theta and p based on the complete data (Y + Z)
        complete_data = Y + Z
        theta = np.sum(complete_data, axis=(0, 1)) / np.sum(complete_data)
        p = np.sum(Y, axis=(0, 1)) / np.sum(complete_data, axis=(0, 1))
        
        # Print progress
        if iter % 10 == 0 or iter == iterations - 1:
            print(f"Iteration {iter + 1}/{iterations}")
            print(f"  Theta: {theta}")
            print(f"  P: {p}\n")
    
    return theta, p

# Run the algorithm
final_theta, final_p = capture_recapture_mcem(Y, theta, p, iterations, sample_size)

print("Final parameter estimates:")
print("  Theta (location probabilities):", final_theta)
print("  P (capture probabilities):", final_p)


Iteration 1/50
  Theta: [0.05072054 0.33981762 0.60946184]
  P: [0.00918635 0.00146908 0.00098294]

Iteration 11/50
  Theta: [0.05035444 0.33249576 0.6171498 ]
  P: [0.00925314 0.00150143 0.00097069]

Iteration 21/50
  Theta: [0.0500882  0.33236263 0.61754917]
  P: [0.00930233 0.00150203 0.00097006]

Iteration 31/50
  Theta: [0.05532998 0.33855293 0.60611708]
  P: [0.00842105 0.00147456 0.00098836]

Iteration 41/50
  Theta: [0.05278397 0.33141412 0.61580191]
  P: [0.00882724 0.00150633 0.00097282]

Iteration 50/50
  Theta: [0.05634506 0.33750458 0.60615036]
  P: [0.00826934 0.00147914 0.00098831]

Final parameter estimates:
  Theta (location probabilities): [0.05634506 0.33750458 0.60615036]
  P (capture probabilities): [0.00826934 0.00147914 0.00098831]


In [2]:
import random

# Simulated data: observed captures (Y) and initial parameters
n = 10  # Number of animals
t = 6   # Number of time points
m = 3   # Number of locations

# Example observed data: random initialization for demonstration
random.seed(42)
Y = [[[random.randint(0, 1) for _ in range(m)] for _ in range(t)] for _ in range(n)]

# Initial parameter guesses
theta = [1.0 / m] * m  # Equal probabilities for locations
p = [0.7 for _ in range(m)]  # Initial capture probabilities

# MCEM algorithm parameters
iterations = 50
sample_size = 1000

def multinomial_sample(probs, size):
    """Generate a multinomial sample."""
    total = sum(probs)
    normalized_probs = [p / total for p in probs]
    counts = [0] * len(probs)
    
    for _ in range(size):
        r = random.random()
        cumulative = 0
        for i, prob in enumerate(normalized_probs):
            cumulative += prob
            if r <= cumulative:
                counts[i] += 1
                break
    
    return counts

def capture_recapture_mcem(Y, theta, p, iterations, sample_size):
    n, t, m = len(Y), len(Y[0]), len(Y[0][0])
    
    for iter in range(iterations):
        # Step 1: Monte Carlo E-step
        # Simulate missing data Z using multinomial distributions
        Z = [[[0] * m for _ in range(t)] for _ in range(n)]
        
        for i in range(n):
            for j in range(t):
                observed = Y[i][j]
                remaining_prob = [(theta[k] * (1 - p[k])) for k in range(m)]
                norm_factor = sum(remaining_prob)
                
                if norm_factor > 0:
                    Z[i][j] = multinomial_sample(remaining_prob, sample_size)
        
        # Step 2: M-step
        # Update theta and p based on the complete data (Y + Z)
        complete_data = [[[Y[i][j][k] + Z[i][j][k] for k in range(m)] for j in range(t)] for i in range(n)]
        
        # Update theta
        theta = [0] * m
        total_complete = sum(sum(sum(row) for row in animal) for animal in complete_data)
        for k in range(m):
            theta[k] = sum(complete_data[i][j][k] for i in range(n) for j in range(t)) / total_complete
        
        # Update p
        p = [0] * m
        for k in range(m):
            observed_captures = sum(Y[i][j][k] for i in range(n) for j in range(t))
            total_complete_k = sum(complete_data[i][j][k] for i in range(n) for j in range(t))
            p[k] = observed_captures / total_complete_k if total_complete_k > 0 else 0
        
        # Print progress
        if iter % 10 == 0 or iter == iterations - 1:
            print(f"Iteration {iter + 1}/{iterations}")
            print(f"  Theta: {theta}")
            print(f"  P: {p}\n")
    
    return theta, p

# Run the algorithm
final_theta, final_p = capture_recapture_mcem(Y, theta, p, iterations, sample_size)

print("Final parameter estimates:")
print("  Theta (location probabilities):", final_theta)
print("  P (capture probabilities):", final_p)


Iteration 1/50
  Theta: [0.3332944955808186, 0.3329449558081859, 0.3337605486109955]
  P: [0.0011985617259288853, 0.0014997750337449383, 0.0012467584280869738]

Iteration 11/50
  Theta: [0.3326120607866309, 0.33264535028878645, 0.33474258892458264]
  P: [0.001201020867737577, 0.0015011258443832875, 0.001243100790612103]

Iteration 21/50
  Theta: [0.3336107458512958, 0.3329449558081859, 0.3334442983405183]
  P: [0.001197425535099536, 0.0014997750337449383, 0.0012479408975190935]

Iteration 31/50
  Theta: [0.33698963032007856, 0.32716922718420743, 0.33584114249571395]
  P: [0.0011854193420922652, 0.0015262515262515263, 0.0012390345442830947]

Iteration 41/50
  Theta: [0.34411358378135454, 0.32647014763894205, 0.3294162685797034]
  P: [0.0011608783979878108, 0.0015295197308045274, 0.0012632004446465565]

Iteration 50/50
  Theta: [0.3520697747965179, 0.32696949017127447, 0.32096073503220757]
  P: [0.0011346444780635401, 0.0015271838729383018, 0.001296478763677851]

Final parameter estimate

In [3]:
import random

# Simulated data: observed captures (Y) and initial parameters
n = 10  # Number of animals
t = 6   # Number of time points
m = 3   # Number of locations

# Example observed data: random initialization for demonstration
random.seed(42)
Y = [[[random.randint(0, 1) for _ in range(m)] for _ in range(t)] for _ in range(n)]

# Initial parameter guesses
theta = [1.0 / m] * m  # Equal probabilities for locations
p = [0.7 for _ in range(m)]  # Initial capture probabilities

# MCEM algorithm parameters
iterations = 50
sample_size = 1000

# To store parameter evolution
theta_history = [[] for _ in range(m)]
p_history = [[] for _ in range(m)]

def multinomial_sample(probs, size):
    """Generate a multinomial sample."""
    total = sum(probs)
    normalized_probs = [p / total for p in probs]
    counts = [0] * len(probs)
    
    for _ in range(size):
        r = random.random()
        cumulative = 0
        for i, prob in enumerate(normalized_probs):
            cumulative += prob
            if r <= cumulative:
                counts[i] += 1
                break
    
    return counts

def capture_recapture_mcem(Y, theta, p, iterations, sample_size):
    n, t, m = len(Y), len(Y[0]), len(Y[0][0])
    
    for iter in range(iterations):
        # Step 1: Monte Carlo E-step
        Z = [[[0] * m for _ in range(t)] for _ in range(n)]
        
        for i in range(n):
            for j in range(t):
                observed = Y[i][j]
                remaining_prob = [(theta[k] * (1 - p[k])) for k in range(m)]
                norm_factor = sum(remaining_prob)
                
                if norm_factor > 0:
                    Z[i][j] = multinomial_sample(remaining_prob, sample_size)
        
        # Step 2: M-step
        complete_data = [[[Y[i][j][k] + Z[i][j][k] for k in range(m)] for j in range(t)] for i in range(n)]
        
        # Update theta
        theta = [0] * m
        total_complete = sum(sum(sum(row) for row in animal) for animal in complete_data)
        for k in range(m):
            theta[k] = sum(complete_data[i][j][k] for i in range(n) for j in range(t)) / total_complete
        
        # Update p
        p = [0] * m
        for k in range(m):
            observed_captures = sum(Y[i][j][k] for i in range(n) for j in range(t))
            total_complete_k = sum(complete_data[i][j][k] for i in range(n) for j in range(t))
            p[k] = observed_captures / total_complete_k if total_complete_k > 0 else 0
        
        # Save history for plotting
        for k in range(m):
            theta_history[k].append(theta[k])
            p_history[k].append(p[k])
        
        # Print progress
        if iter % 10 == 0 or iter == iterations - 1:
            print(f"Iteration {iter + 1}/{iterations}")
            print(f"  Theta: {theta}")
            print(f"  P: {p}\n")
    
    return theta, p

# Run the algorithm
final_theta, final_p = capture_recapture_mcem(Y, theta, p, iterations, sample_size)

# Function to plot ASCII graphs
def plot_ascii(data, label, iterations):
    max_value = max(max(history) for history in data)
    min_value = min(min(history) for history in data)
    height = 10
    width = iterations
    scale = height / (max_value - min_value) if max_value != min_value else 1
    
    print(f"\n{label} over iterations:")
    for k, history in enumerate(data):
        print(f"  Parameter {k + 1}:")
        for y in range(height, -1, -1):
            line = ""
            for x in range(width):
                value = (history[x] - min_value) * scale
                line += "*" if int(value) == y else " "
            print(line)
        print("-" * width)

# Plot results
plot_ascii(theta_history, "Theta", iterations)
plot_ascii(p_history, "P", iterations)

print("Final parameter estimates:")
print("  Theta (location probabilities):", final_theta)
print("  P (capture probabilities):", final_p)


Iteration 1/50
  Theta: [0.3332944955808186, 0.3329449558081859, 0.3337605486109955]
  P: [0.0011985617259288853, 0.0014997750337449383, 0.0012467584280869738]

Iteration 11/50
  Theta: [0.3326120607866309, 0.33264535028878645, 0.33474258892458264]
  P: [0.001201020867737577, 0.0015011258443832875, 0.001243100790612103]

Iteration 21/50
  Theta: [0.3336107458512958, 0.3329449558081859, 0.3334442983405183]
  P: [0.001197425535099536, 0.0014997750337449383, 0.0012479408975190935]

Iteration 31/50
  Theta: [0.33698963032007856, 0.32716922718420743, 0.33584114249571395]
  P: [0.0011854193420922652, 0.0015262515262515263, 0.0012390345442830947]

Iteration 41/50
  Theta: [0.34411358378135454, 0.32647014763894205, 0.3294162685797034]
  P: [0.0011608783979878108, 0.0015295197308045274, 0.0012632004446465565]

Iteration 50/50
  Theta: [0.3520697747965179, 0.32696949017127447, 0.32096073503220757]
  P: [0.0011346444780635401, 0.0015271838729383018, 0.001296478763677851]


Theta over iterations:


### EM Standard Errors

There are various algorithms and formulas available for obtaining standard errors from the EM algorithm (see Tanner, 1996, for a selection). However, the formulation by Oakes (1999) and its Monte Carlo version seem both simple and useful.

Recall that the variance of the MLE, \( \hat{\theta} \), is approximated by:

\[
\text{Var}(\hat{\theta}) \approx -\mathbb{E}\left[\frac{\partial^2}{\partial \theta^2} \log L(\theta \mid x)\right].
\]

Oakes (1999) shows that this second derivative can be expressed in terms of the complete-data likelihood:

\[
\frac{\partial^2}{\partial \theta^2} \log L(\theta \mid x) = 
\mathbb{E} \left[\frac{\partial^2}{\partial \theta^2} \log L(\theta \mid x, z)\right] +
\text{Var}\left[\frac{\partial}{\partial \theta} \log L(\theta \mid x, z)\right],
\]

where the expectation is taken with respect to the distribution of the missing data. Thus, for the EM algorithm, we have a formula for the variance of the MLE.

---

### Advantages and Challenges

The advantage of this expression is that it only involves the distribution of the complete data, which is often a reasonable distribution to work with. However, the mixed derivative may be difficult to compute. In terms of complexity, it compares favorably with other methods.

---

### Monte Carlo EM and Oakes' Identity

For the Monte Carlo EM algorithm, the expression above cannot be used in its current form, as we want all expectations to be on the outside. By taking the derivative inside the expectation, Oakes' identity can be rewritten as:

\[
\frac{\partial^2}{\partial \theta^2} \log L(\theta \mid x) =
\mathbb{E}\left[\frac{\partial^2}{\partial \theta^2} \log L(\theta \mid x, z)\right] -
\mathbb{E}\left[\left(\frac{\partial}{\partial \theta} \log L(\theta \mid x, z)\right)^2\right] +
\mathbb{E}\left[\left(\frac{\partial}{\partial \theta} \log L(\theta \mid x, z)\right)\right]^2.
\]

This formulation is better suited for simulation since all expectations are now on the outside.

---

### Simplified Representation

Equation (5.22) can be expressed in the rather pleasing form:

\[
\frac{\partial^2}{\partial \theta^2} \log L(\theta \mid x) =
\mathbb{E}\left[\frac{\partial^2}{\partial \theta^2} \log L(\theta \mid x, z)\right] -
\text{Var}\left[\frac{\partial}{\partial \theta} \log L(\theta \mid x, z)\right].
\]

This expression facilitates the computation of standard errors in Monte Carlo EM methods while leveraging the distribution of the complete data.


### Genetic Linkage Standard Errors

The second derivative of the log-likelihood, as derived in the EM framework, can be expressed as:

\[
\frac{\partial^2}{\partial \theta^2} \log L(x) = 
\mathbb{E}\left[\frac{\partial^2}{\partial \theta^2} \log L(\theta \mid x, z)\right] + 
\text{Var}\left[\frac{\partial}{\partial \theta} \log L(\theta \mid x, z)\right] - 
\text{Var}\left(\log L(\theta \mid x, z)\right).
\]

For Monte Carlo evaluation, this becomes:

\[
\frac{\partial^2}{\partial \theta^2} \log L(x) \approx 
\frac{1}{M} \sum_{j=1}^M \frac{\partial^2}{\partial \theta^2} \log L(\theta \mid x, z_j) - 
\frac{1}{M} \left[\sum_{j=1}^M \frac{\partial}{\partial \theta} \log L(\theta \mid x, z_j)\right]^2 +
\frac{1}{M} \sum_{j=1}^M \left[\frac{\partial}{\partial \theta} \log L(\theta \mid x, z_j)\right]^2,
\]

where \( z_j \), \( j = 1, \dots, M \), are generated from the missing data distribution and have already been generated to perform MCEM.

---

### Example 5.23: Genetic Linkage Standard Errors

For Example 5.21, the complete-data likelihood is given by:

\[
L(x) = \theta^{x_1 + x_2} (1 - \theta)^{x_3 + x_4},
\]

and applying the variance formula (5.22), we obtain:

\[
\frac{\partial^2}{\partial \theta^2} \log L(x) = 
\frac{-x_1}{\theta^2} + \frac{-x_2}{(1 - \theta)^2} + \frac{(x_3 + x_4)A^2}{\theta(1 - \theta)},
\]

where \( A = \mathbb{E}\left[\frac{\theta}{\theta + 1}\right] \). 

---

### Practical Evaluation

In practice, we evaluate the expectation at the converged value of the likelihood estimator. For these data, we obtain:

\[
\hat{\theta} = 0.627 \quad \text{with a standard deviation of } 0.048.
\]

---

### Visualization

The figure below illustrates the evolution of the EM estimate and its associated standard error over 25 iterations.

---

```python
# Pseudo-code for plotting the graph in Jupyter Notebook
import matplotlib.pyplot as plt

iterations = list(range(1, 26))
theta_estimates = [0.6 + 0.027 * (i / 25) for i in iterations]  # Simulated EM sequence
std_deviation = [0.05 - 0.002 * (i / 25) for i in iterations]  # Simulated standard deviation sequence

plt.figure(figsize=(10, 6))
plt.plot(iterations, theta_estimates, label="EM Estimate (θ)", color="blue")
plt.fill_between(iterations, 
                 [t - s for t, s in zip(theta_estimates, std_deviation)], 
                 [t + s for t, s in zip(theta_estimates, std_deviation)], 
                 color="blue", alpha=0.2, label="± 1 Standard Deviation")

plt.xlabel("Iteration")
plt.ylabel("Estimate")
plt.title("EM Sequence and Standard Deviation for Genetic Linkage Data")
plt.legend()
plt.grid()
plt.show()


# 5.5 Notes

## 5.5.1 Variations on EM

The EM algorithm is a powerful optimization tool, but it has its limitations. Notably:

1. **Difficult Computation in the E-step**: 
   - As discussed in Section 5.3.3, the E-step can become computationally challenging for complex likelihoods.
2. **Multimodal Likelihoods**: 
   - EM converges to the maximum likelihood estimator for unimodal likelihoods but may get trapped in local maxima for multimodal likelihoods, making the results dependent on initial conditions.

### Overcoming Initial Condition Dependence

Several variations of EM have been proposed to address these issues. Below, we describe some key methods:

---

### **Stochastic EM (SEM)**

Broniatowski et al. (1984) and Celeux and Diebolt (1985, 1992) introduced the **Stochastic EM (SEM)** algorithm, which modifies the standard EM algorithm:

1. **Step 1**: Instead of computing the expected value, simulate the missing data \( z \) conditionally on the observations \( x \) and the current parameter value \( \theta^{(m)} \).
2. **Step 2**: Maximize the complete-data log-likelihood \( H(x, z_m \mid \theta) \).

This stochastic step allows SEM to explore the likelihood surface more systematically, reducing its susceptibility to being trapped in local modes.

---

#### **Convergence Challenges**

- **Ergodicity**: The Markov chain \( \theta^{(m)} \) generated by SEM is often ergodic, but the relationship between the stationary distribution and the maxima of the observed likelihood is rarely clear.
- **Ergodic Average**: 
  \[
  \bar{\theta}^{(M)} = \frac{1}{M} \sum_{m=1}^M \theta^{(m)},
  \]
  is studied instead of the global mode:
  \[
  \hat{\theta}^{(M)} = \arg \max_\theta L(\theta).
  \]

---

### **Simulated Annealing EM (SAEM)**

To address the convergence problem in SEM, Celeux and Diebolt (1990) introduced **SAEM**:

- Gradually reduce the randomness in the simulations during iterations.
- End with a standard EM algorithm.

SAEM connects to **Simulated Annealing** methods (see Section 5.2.3) and achieves convergence to the global mode under specific conditions.

---

### **Hybrid Approaches**

1. **Celeux et al. (1996)**:
   - Combine SEM and EM by using SEM to generate the starting point for EM.
   
2. **Lavielle and Moulines (1997)**:
   - Similar to Celeux and Diebolt (1990), they propose convergence conditions equivalent to EM.

3. **SAME Algorithm** (Doucet et al., 2002):
   - **State Augmentation for Marginal Estimation (SAME)**:
     - Integrates ideas from Sections 5.2.4 and 5.3.1 within an MCMC framework.

---

### **Expectation Conditional Maximization (ECM)**

Meng and Rubin (1991, 1992), Liu and Rubin (1994), and Meng and van Dyk (1997) developed **ECM**, which takes advantage of Gibbs sampling:

- Maximizes the complete-data likelihood along successive directions (conditional likelihoods).
- This approach reduces the computational complexity of the E-step by focusing on subsets of parameters.

---

These methods enhance the robustness and flexibility of the EM algorithm, enabling it to handle more complex likelihood surfaces and data structures.



In [4]:
import numpy as np
import matplotlib.pyplot as plt

# Step 1: Generate synthetic data (observed + missing values)
np.random.seed(42)

# Assume we have 100 data points and a parameter theta
theta_true = 0.7
n_data = 100

# Simulate data (binary outcome)
observed_data = np.random.binomial(1, theta_true, n_data)

# Simulate missingness (randomly mask some data points as missing)
missingness = np.random.rand(n_data) < 0.3  # 30% missing data
observed_data[missingness] = np.nan  # Mark missing data as NaN

# Step 2: Define the E-step (simulate missing data using conditional distribution)
def e_step(observed_data, theta):
    """ Simulate missing data using conditional distribution. """
    simulated_data = np.copy(observed_data)
    missing_indices = np.isnan(observed_data)
    
    # Simulate missing data (conditional on theta)
    simulated_data[missing_indices] = np.random.binomial(1, theta, np.sum(missing_indices))
    
    return simulated_data

# Step 3: Define the M-step (maximize the likelihood)
def m_step(completed_data):
    """ Maximize the likelihood to estimate theta. """
    # Maximum likelihood estimate for a binomial distribution: the sample mean
    theta_estimate = np.nanmean(completed_data)  # Ignore NaNs when calculating the mean
    return theta_estimate

# Step 4: Run the SEM algorithm
def sem_algorithm(observed_data, max_iter=100, theta_init=0.5):
    theta = theta_init
    theta_history = [theta]  # Keep track of theta estimates for plotting
    
    for iteration in range(max_iter):
        # E-step: simulate missing data
        simulated_data = e_step(observed_data, theta)
        
        # M-step: maximize the likelihood (estimate theta)
        theta = m_step(simulated_data)
        
        # Track the theta estimates
        theta_history.append(theta)
    
    return theta_history

# Run the SEM algorithm
theta_estimates = sem_algorithm(observed_data, max_iter=100)

# Plot the convergence of theta estimates
plt.plot(theta_estimates, label='Theta Estimates')
plt.axhline(y=theta_true, color='r', linestyle='--', label='True Theta')
plt.xlabel('Iteration')
plt.ylabel('Theta Estimate')
plt.legend()
plt.title('Convergence of Stochastic EM Algorithm')
plt.show()


ValueError: cannot convert float NaN to integer

In [5]:
import random

# Step 1: Generate synthetic data (observed + missing values)
random.seed(42)

# Assume we have 100 data points and a parameter theta
theta_true = 0.7
n_data = 100

# Simulate data (binary outcome)
observed_data = []
for i in range(n_data):
    observed_data.append(1 if random.random() < theta_true else 0)

# Simulate missingness (randomly mask some data points as missing)
missingness = [random.random() < 0.3 for _ in range(n_data)]  # 30% missing data
for i in range(n_data):
    if missingness[i]:
        observed_data[i] = None  # Mark missing data as None

# Step 2: Define the E-step (simulate missing data using conditional distribution)
def e_step(observed_data, theta):
    """ Simulate missing data using conditional distribution. """
    simulated_data = observed_data[:]
    
    for i in range(len(observed_data)):
        if simulated_data[i] is None:
            # Simulate missing data (conditional on theta)
            simulated_data[i] = 1 if random.random() < theta else 0
    
    return simulated_data

# Step 3: Define the M-step (maximize the likelihood)
def m_step(completed_data):
    """ Maximize the likelihood to estimate theta. """
    # Maximum likelihood estimate for a binomial distribution: the sample mean
    count = 0
    total = 0
    for data_point in completed_data:
        if data_point is not None:
            total += data_point
            count += 1
    
    theta_estimate = total / count if count > 0 else 0  # Avoid division by zero
    return theta_estimate

# Step 4: Run the SEM algorithm
def sem_algorithm(observed_data, max_iter=100, theta_init=0.5):
    theta = theta_init
    theta_history = [theta]  # Keep track of theta estimates for plotting
    
    for iteration in range(max_iter):
        # E-step: simulate missing data
        simulated_data = e_step(observed_data, theta)
        
        # M-step: maximize the likelihood (estimate theta)
        theta = m_step(simulated_data)
        
        # Track the theta estimates
        theta_history.append(theta)
    
    return theta_history

# Run the SEM algorithm
theta_estimates = sem_algorithm(observed_data, max_iter=100)

# Show the convergence of theta estimates
for iteration, theta in enumerate(theta_estimates):
    print(f"Iteration {iteration}: Theta Estimate = {theta}")

# Final estimated theta after all iterations
print(f"\nFinal Theta Estimate after {len(theta_estimates)-1} iterations: {theta_estimates[-1]}")


Iteration 0: Theta Estimate = 0.5
Iteration 1: Theta Estimate = 0.65
Iteration 2: Theta Estimate = 0.71
Iteration 3: Theta Estimate = 0.75
Iteration 4: Theta Estimate = 0.65
Iteration 5: Theta Estimate = 0.73
Iteration 6: Theta Estimate = 0.79
Iteration 7: Theta Estimate = 0.79
Iteration 8: Theta Estimate = 0.78
Iteration 9: Theta Estimate = 0.8
Iteration 10: Theta Estimate = 0.81
Iteration 11: Theta Estimate = 0.74
Iteration 12: Theta Estimate = 0.74
Iteration 13: Theta Estimate = 0.8
Iteration 14: Theta Estimate = 0.77
Iteration 15: Theta Estimate = 0.78
Iteration 16: Theta Estimate = 0.77
Iteration 17: Theta Estimate = 0.76
Iteration 18: Theta Estimate = 0.74
Iteration 19: Theta Estimate = 0.73
Iteration 20: Theta Estimate = 0.73
Iteration 21: Theta Estimate = 0.76
Iteration 22: Theta Estimate = 0.76
Iteration 23: Theta Estimate = 0.7
Iteration 24: Theta Estimate = 0.75
Iteration 25: Theta Estimate = 0.77
Iteration 26: Theta Estimate = 0.78
Iteration 27: Theta Estimate = 0.78
Iterat

In [6]:
def plot_progress(theta_history):
    """ Plot the progress of theta estimates as a text-based plot. """
    # Determine the range of theta values for scaling the plot
    min_theta = min(theta_history)
    max_theta = max(theta_history)
    
    # Define the height of the graph
    graph_height = 20
    graph_width = len(theta_history)
    
    # Scale the theta values into the graph height
    scaled_theta = [
        int((theta - min_theta) / (max_theta - min_theta) * (graph_height - 1))
        for theta in theta_history
    ]
    
    # Print the graph
    for i in range(graph_height - 1, -1, -1):
        line = ""
        for value in scaled_theta:
            if value >= i:
                line += "█"
            else:
                line += " "
        print(line)
    
    # Print the base of the graph
    print("-" * graph_width)
    print(f"Iterations: {len(theta_history)}")
    
# Run the SEM algorithm to get theta estimates
theta_estimates = sem_algorithm(observed_data, max_iter=100)

# Plot the progress of theta estimates
plot_progress(theta_estimates)


                                █            █                                                       
              █            █    █            █                                                       
   █    ██ █████   █ ██    █ █  █       ███  ██   █  █   █         █       ████  █          ██  █  █ 
   █    ████████   ██████  █ █  █ █    ████  ███ ██  ██  █        ███     █████  █         ████ █ ███
   █ ███████████  ████████ █ ██████   ██████ ███ ██████  ██  █ █  ███ █ █ ████████     █   ██████████
   █ ███████████████████████████████ ███████ ████████████████████████ █████████████    █ ████████████
   █████████████████████████████████ ██████████████████████████████████████████████   ███████████████
   █████████████████████████████████████████████████████████████████████████████████  ███████████████
   ██████████████████████████████████████████████████████████████████████████████████ ███████████████
   ███████████████████████████████████████████████████████████████████████████████

##  Neural Networks

Neural networks provide another type of missing data model where simulation methods are almost always necessary. These models are frequently used in classification and pattern recognition, as well as in robotics and computer vision (see Cheng and Titterington 1994 for a review on these models). Barring the biological vocabulary and the idealistic connection with actual neurons, the theory of neural networks covers:

(i) modeling nonlinear relations between explanatory and dependent (explained) variables,  
(ii) estimation of the parameters of these models based on a (training) sample.

Although the neural network literature usually avoids probabilistic modeling, these models can be analyzed and estimated from a statistical point of view (see Neal 1999 or Ripley 1994, 1996). They can also be seen as a particular type of nonparametric estimation problem, where a major issue is then identifiability.

A simple classical example of a neural network is the multilayer model (also called the backpropagation model) which relates explanatory variables $ \mathbf{x} = (x_1, \ldots, x_p) $ with dependent variables $ \mathbf{y} = (y_1, \ldots, y_n) $ through a hidden "layer", $ \mathbf{h} = (h_1, \ldots, h_p) $, where $ (k = 1, \ldots, p; i = 1, \ldots, n) $:

$$
h_k = f \left( \alpha_k + \sum_{j=1}^p a_{kj} x_j \right)
$$

$$
E[\mathbf{y}_i | \mathbf{h}] = g(\mathbf{B} \mathbf{h} + \mathbf{c})
$$

and 

$$
\text{Var}(\mathbf{y}) = \sigma^2.
$$

The functions $ f $ and $ g $ are known (or arbitrarily chosen) from categories such as threshold, $ f(t) = \mathbb{I}(t > 0) $, hyperbolic, $ f(t) = \tanh(t) $, or sigmoid, $ f(t) = \frac{1}{1 + e^{-t}} $.

As an example, consider the problem of character recognition, where handwritten manuscripts are automatically deciphered. The $ x_i $'s may correspond to geometric characteristics of a digitized character or pixel gray levels, and the $ y_i $'s are the 26 letters of the alphabet, plus side symbols. (See Le Cun et al. 1989 for an actual modeling based on a sample of 7291, 16 x 16 pixel images, for 9760 parameters.)

The likelihood of the multilayer model then includes the parameters $ \alpha = (\alpha_k) $ and $ \beta = (\beta_k) $ in a nonlinear structure. Assuming normality, for observations $ (r, t) $, $ t = 1, 2, \ldots, T $, the log-likelihood can be written:

$$
\ell(\alpha, \beta | \mathbf{x}, \mathbf{y}) = -\sum_{i=1}^T \frac{(y_i - f(\mathbf{x}_i))^2}{2 \sigma^2}
$$

A similar objective function can be derived using a least squares criterion. The maximization of $ f(\mathbf{x}, \beta | \mathbf{x}, \mathbf{y}) $ involves the detection and elimination of numerous local modes.

---

##  The Robbins-Monro Procedure

The Robbins-Monro algorithm (Robbins and Monro 1951) is a technique of stochastic approximation to solve for $ x $ in equations of the form:

$$
x_{n+1} = x_n + \alpha_n \left( g(x_n) - x_n \right)
$$

where $ g(x) $ is a function for which we need to find the root, and $ \alpha_n $ is a step size that typically decreases as $ n $ increases.
## Robbins-Monro Procedure and Convergence

The Robbins-Monro method proceeds by generating a Markov chain of the form:

$$
X_{j+1} = X_j + \alpha_j \left( h(Z_j, X_j) \right),
$$

where $ Z $ is simulated from the conditional distribution defining $ h(x) $ as in equation (5.8). The following result describes sufficient conditions for the algorithm to be convergent (see Bouleau and Lépingle 1994 for a proof).

### Theorem 5.24

If $\{\alpha_n\} $is a sequence of positive numbers such that:

$$
\sum_{n=1}^\infty \alpha_n = \infty \quad \text{and} \quad \sum_{n=1}^\infty \alpha_n^2 < \infty,
$$

and if the $ Z_n $'s are simulated from $ H $ conditionally on $ \theta $, such that:

$$
\mathbb{E}[h(X_n, \theta)] = h(\theta),
$$

and $ |Z_n| \leq B $ for a fixed bound $ B $, and if there exists $ \theta^* $ such that:

$$
\inf_{\theta \neq \theta^*} \left( \theta - \theta^* \right) \cdot \left( h(\theta) - h(\theta^*) \right) > 0,
$$

then the sequence $ \{ \theta_n \} $ converges to $ \theta^* $ almost surely.

---

### Solution of the Maximization Problem

The solution of the maximization problem can be expressed in terms of the solution of the equation $ \nabla h(\theta) = 0 $ if the problem is sufficiently regular, i.e., if the maximum is not achieved on the boundary of the domain $ \Theta $.

Note that the equation is similar to (5.24). Since its proposal, this method has seen numerous variations. For more detailed references, see Benveniste et al. (1990), Bouleau and Lépingle (1994), Wasan (1969), Kersting (1987), Winkler (1995), and Duflo (1996).

### Case with Several Local Maxima

When the function $ h $ has several local maxima, the Robbins-Monro procedure converges to one of these maxima. In the particular case where $ H(x, \theta) = h(\theta) + \frac{x}{\sqrt{a}} $ (with $ a > 0$), Pflug (1994) examines the relation:

$$
\theta_{s+1} = \theta_s + a h(\theta) + \sqrt{a}/2 x,
$$

where the $ X_n $'s are i.i.d. with $ \mathbb{E}[X] = 0 $ and $ \text{Var}(X) = T $.

The relevance of this particular case is that, under the following conditions:

1. $ h(\theta) > 0 $, $ k > 0 $ such that $ -h(\theta) \leq k \|\theta\| $ for $ \theta > k $,
2. $ |h(\theta)| \leq k h(\theta^*) $ for $ k < 1000 $,
3. $ h'(\theta) > k s $,

the stationary measure associated with the Markov chain weakly converges to the distribution with density:

$$
c(T) \exp\left( \frac{h(\theta)}{T} \right),
$$

where $ c(T) $ is a normalizing constant.
## Robbins-Monro Convergence and Gibbs Measure

When $ a $ goes to 0, and $ T $ remains fixed, with $ E $ being the primitive of $ h $ (see Duflo 1996), the hypotheses (i)-(iii) ensure that:

$$
\exp\left(\frac{E(\theta)}{T}\right)
$$

is integrable on $ \mathbb{R} $. Therefore, the limiting distribution of $ \theta $ (as $ a \to 0 $) is the so-called **Gibbs measure** with energy function $ E $, which is a pivotal quantity for the **simulated annealing** method introduced in Section 5.2.3. 

In particular, when $ T $ goes to 0, the Gibbs measure converges to the uniform distribution on the set of (global) maxima of $ E $ (see Hwang 1980). This convergence is interesting more because of the connections it exhibits with the notions of **Gibbs measure** and **simulated annealing**, rather than for its practical consequences.

The assumptions (i)-(iii) are rather restrictive and difficult to check for implicit $ h $'s, and the representation in equation (5.25) is quite specialized. Note, however, that the completion of $ h() $ in $ H(x, \theta) $ is free since the conditions (i)-(iii) relate only to $ h $.

---

## Monte Carlo Approximation

In cases where a function $ h(z) $ can be written as $ \mathbb{E}[H(z, Z)] $ but is not directly computable, it can be approximated by the empirical (Monte Carlo) average:

$$
h(z) \approx \frac{1}{M} \sum_{i=1}^{M} H(x, Z_i),
$$

where the $ Z_i $'s are generated from the conditional distribution $ f(x) $. This approximation yields a convergent estimator of $ h(z) $ for every value of $ x $, but its use in optimization setups is not recommended for at least two related reasons:

1. Presumably, $ h(x) $ needs to be evaluated at many points, which will involve the generation of many samples of $ Z $'s of size $ M $.
2. Since the sample changes with every value of $ x $, the resulting sequence of evaluations of $ h $ will usually not be smooth.

These difficulties prompted Geyer (1996) to suggest an **importance sampling** approach to this problem, using a single sample of $ Z $'s simulated from $g(Z) $ and estimating $ h(x) $ with:

$$
h_m(x) = \frac{1}{M} \sum_{i=1}^{M} H(x, Z_i) \frac{f(Z_i)}{g(Z_i)},
$$

where the $ Z_i $'s are simulated from $ g(Z) $. Since this evaluation of $ h $ does not depend on $ x $, points (i) and (ii) above are answered.

The problem then shifts from the original maximization problem to:

$$
\max_x h_m(x),
$$

which leads to a convergent solution in most cases and also allows for the use of regular optimization techniques, since the function $ h_m $ does not vary with each iteration. 

However, three remaining drawbacks of this approach are as follows:

1. Since $ h_m $ is expressed as a sum, it most often enjoys fewer analytical properties than the original function $ h $.
2. The choice of the importance function $ g $ can be very influential in obtaining a good approximation of the function $ h(x) $.


In [7]:
import random
import math

# Define the conditional distribution f(x), for example, a normal distribution
def conditional_distribution(x):
    # Simulating a conditional distribution, f(x), for example a normal distribution
    # Here we assume standard normal distribution for simplicity
    return random.gauss(x, 1)  # mean = x, standard deviation = 1

# Define the function H(x, Z), which depends on both x and the random variable Z
def H(x, Z):
    # Example: H(x, Z) = (x - Z)^2
    return (x - Z) ** 2

# Monte Carlo Approximation to compute h(x)
def monte_carlo_approximation(x, M):
    # Generate M samples from the conditional distribution f(x)
    samples = [conditional_distribution(x) for _ in range(M)]
    
    # Calculate the Monte Carlo average of H(x, Z)
    h_approx = sum(H(x, Z) for Z in samples) / M
    return h_approx

# Example usage
x = 5  # Evaluate at x = 5
M = 1000  # Number of samples for Monte Carlo approximation
h_approx = monte_carlo_approximation(x, M)
print(f"Monte Carlo approximation of h({x}): {h_approx}")
# Define the function h(x), for example h(x) = x^2
def h(x):
    return x ** 2

# Robbins-Monro procedure
def robbins_monro(B, initial_x, alpha, iterations):
    x = initial_x
    for _ in range(iterations):
        # Simulate Z from the distribution of Z given x, here we use a simple Gaussian random walk
        Z = random.gauss(0, 1)
        
        # Update x using the Robbins-Monro update rule
        x = x + alpha * (B - h(x)) + Z
        
    return x

# Example usage
B = 25  # We want to solve x^2 = B, so B = 25
initial_x = 1.0  # Initial guess for x
alpha = 0.1  # Step size for updates
iterations = 100  # Number of iterations to run

estimated_x = robbins_monro(B, initial_x, alpha, iterations)
print(f"Estimated solution using Robbins-Monro: x ≈ {estimated_x}")


Monte Carlo approximation of h(5): 0.9591290923449101
Estimated solution using Robbins-Monro: x ≈ 5.521838692236382
